In [1]:
import sqlite3 as sql
import numpy as np

# Problem 1
def prob1(db_file="students.db"):
    """Query the database for the list of the names of students who have a
    'B' grade in any course. Return the list.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): a list of strings, each of which is a student name.
    """
    try:
        with sql.connect(db_file) as conn:
            cur = conn.cursor()

            names = cur.execute("SELECT SI.StudentName "
                        "FROM StudentInfo AS SI INNER JOIN StudentGrades AS SG "
                        "ON SI.STudentID == SG.StudentID "
                        "WHERE SG.Grade == 'B';").fetchall()       #use an inner search to match the student id columns of the two tables and then find the students who have B's in a class
    finally:
        conn.close()

    return list(np.ravel(names))         #return a list of names

def test1():
    print(prob1())
test1()

['Gilbert Chapman', 'Mercedes Hall', 'Alfonso Phelps']


In [2]:
# Problem 2
def prob2(db_file="students.db"):
    """Query the database for all tuples of the form (Name, MajorName, Grade)
    where 'Name' is a student's name and 'Grade' is their grade in Calculus.
    Only include results for students that are actually taking Calculus, but
    be careful not to exclude students who haven't declared a major.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """

    try:
        with sql.connect(db_file) as conn:      #open connection
            cur = conn.cursor()

            tuples = cur.execute("SELECT SI.StudentName, MI.MajorName, SG.Grade "
                                "FROM StudentInfo AS SI LEFT OUTER JOIN MajorInfo AS MI "   #outer join student info and major info
                                "ON SI.MajorID == MI.MajorID "
                                "INNER JOIN CourseInfo as CI, StudentGrades AS SG "      #two inner joins with course info student grades
                                "ON SI.StudentID == SG.StudentID AND SG.CourseID == CI.CourseID "
                                "WHERE CI.CourseName == 'Calculus';").fetchall()       #find all the students in calculus

    finally:
        conn.close()

    return tuples

def test2():
    print(prob2())
test2()

[('Kristopher Tran', 'Science', 'C+'), ('Cassandra Holland', 'Math', 'A-'), ('Sammy Burke', 'Science', 'A'), ('Gilbert Chapman', None, 'B'), ('Mercedes Hall', None, 'B'), ('Rene Cross', 'Writing', 'A')]


In [3]:
# Problem 3
def prob3(db_file="students.db"):
    """Query the database for the list of the names of courses that have at
    least 5 students enrolled in them.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): a list of strings, each of which is a course name.
    """
    try:
        with sql.connect(db_file) as conn:      #open connection
            cur = conn.cursor()

            classes = cur.execute("SELECT CI.CourseName "
                                "FROM StudentInfo AS SI " 
                                "INNER JOIN CourseInfo AS CI, StudentGrades SG "   #inner join studentinfo and courseinfo and studentgrades and courseinfo
                                "ON SI.StudentID == SG.StudentID AND CI.CourseID==SG.CourseID "
                                "GROUP BY CI.CourseName "
                                "HAVING COUNT(*) >= 5;").fetchall()          #count the number of the same coursenames that appear in the table five or more times

    finally:
        conn.close()
    
    return list(np.ravel(classes))       #return as list

def test3():
    print(prob3())
test3()

['Calculus', 'English']


In [4]:
# Problem 4
def prob4(db_file="students.db"):
    """Query the given database for tuples of the form (MajorName, N) where N
    is the number of students in the specified major. Sort the results in
    descending order by the counts N, then in alphabetic order by MajorName.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    try:
        with sql.connect(db_file) as conn:      #open connection
            cur = conn.cursor()

            majors = cur.execute("SELECT MI.MajorName, COUNT(*) AS num_students " 
                                "FROM StudentInfo AS SI LEFT OUTER JOIN MajorInfo AS MI "     #outer join student info and major info on majorid
                                "ON MI.MajorID == SI.MajorID "
                                "GROUP BY MI.MajorName "                          #group major names by number of occurrences and then alphabetically
                                "ORDER BY num_students DESC, MI.MajorName ASC;").fetchall()          

    finally:
        conn.close()

    return majors
prob4()

[(None, 3), ('Science', 3), ('Math', 2), ('Art', 1), ('Writing', 1)]

In [5]:
# Problem 5
def prob5(db_file="students.db"):
    """Query the database for tuples of the form (StudentName, MajorName) where
    the last name of the specified student begins with the letter C.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    try:
        with sql.connect(db_file) as conn:      #open connection
            cur = conn.cursor()

            majors = cur.execute("SELECT SI.StudentName, MI.MajorName "              #find all the names and majors of students with last name starting in C
                                "FROM StudentInfo AS SI LEFT OUTER JOIN MajorInfo AS MI "
                                "ON SI.MajorID == MI.MajorID "
                                "Where SI.StudentName LIKE '% C%';").fetchall()       

    finally:
        conn.close()

    return majors
prob5()

[('Gilbert Chapman', None),
 ('Roberta Cook', 'Science'),
 ('Rene Cross', 'Writing')]

In [6]:
# Problem 6
def prob6(db_file="students.db"):
    """Query the database for tuples of the form (StudentName, N, GPA) where N
    is the number of courses that the specified student is in and 'GPA' is the
    grade point average of the specified student according to the following
    point system.

        A+, A  = 4.0    B  = 3.0    C  = 2.0    D  = 1.0
            A- = 3.7    B- = 2.7    C- = 1.7    D- = 0.7
            B+ = 3.4    C+ = 2.4    D+ = 1.4

    Order the results from greatest GPA to least.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    try:
        with sql.connect(db_file) as conn:      #open connection
            cur = conn.cursor()

            majors = cur.execute("SELECT SI.StudentName, COUNT(*), AVG(CASE SG.Grade "      #calculate the average grade from our scale
                                        "WHEN 'A+' THEN 4.0 "
                                        "WHEN 'A' THEN 4.0 "        #create our gpa score database
                                        "WHEN 'A-' THEN 3.7 "
                                        "WHEN 'B+' THEN 3.4 "
                                        "WHEN 'B' THEN 3.0 "
                                        "WHEN 'B-' THEN 2.7 "
                                        "WHEN 'C+' THEN 2.4 "
                                        "WHEN 'C' THEN 2.0 "
                                        "WHEN 'C-' THEN 1.7 "
                                        "WHEN 'D+' THEN 1.4 "
                                        "WHEN 'D' THEN 1.0 "
                                        "WHEN 'D-' THEN 0.7 "
                                        "ELSE 0 END) AS gpa "
                                    "FROM StudentGrades AS SG "
                                "INNER JOIN StudentInfo AS SI "         #inner join grades and info on student id
                                "ON SG.StudentID == SI.StudentID "
                                "GROUP BY SG.StudentID "
                                "ORDER BY gpa DESC;").fetchall()        #order in descending order of gpa

    finally:
        conn.close()

    return majors
prob6()

[('Rene Cross', 1, 4.0),
 ('Sammy Burke', 2, 3.85),
 ('Gilbert Chapman', 3, 3.6666666666666665),
 ('Kristopher Tran', 2, 3.2),
 ('Mercedes Hall', 3, 2.6666666666666665),
 ('Cameron Kim', 3, 2.566666666666667),
 ('Alfonso Phelps', 2, 2.5),
 ('Michelle Fernandez', 2, 2.35),
 ('Cassandra Holland', 2, 2.2),
 ('Roberta Cook', 1, 2.0)]